### Molecular Dynamics Steps
##### <span style="color:blue">Step 1: Prepare</span>
##### Step 2: Relax/minimize
##### Step 3: Equilibrate
##### Step 4: Production



# Prepare System
###### We will use the following resources: tleap (part of AMBERTOOLS), and PyMol.

### 1. Find/make initial structure of your system 

<p>Once you know what system you want to simulate, the easiest way to set up the system is to find a crystal structure representative of the system you are interested in. 

The Protein Data Bank is a database with collections of crystal structures:
https://www.rcsb.org 

You will want your initial structure file to be in the pdb format: https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html </p>


<p>Things to consider:
    
    *most pdb files will required modifications*

1. Do you want to simulate a system that has a ligand bound or unbound? If you have a ligand you will need to include additional force field parameters in following steps. A number of small organic molecules have been previously parameterized, so you will have to see if that is the case for your ligand. If it is not, you will need to develop the parameters. There are numerous methods to do this. I have used RESP before, but I'm not totally convinced it's the best way. Either way, if you parameterize something you should include that information in your SI!!! 
    
    RESP: [Bayly:93:10269-10280] C. I. Bayly et. al. J. Phys. Chem. 97, 10269 (1993)
    
    a) if you are keeping a ligand, you should save the ligand coordinates in SDF format, and delete it from the PDB file. At this point you need to separate protein and non-protein molecules (except ions and water) because they will get different treatment in later steps. 
    
    

2. Is there a mutation in the crystal structure? Pretty common. Easy fix, just delete the atoms that are not in the residue you want to be there, and change the three letter residue ID. A program used in later steps will be able to identify there are missing atoms in that residue and place them. 

    
3. Are there missing residues or heavy atoms? Probably, but it is only an issue if there are consecutive missing residues. There are programs that exist to try and predict positions of missing residues if this is the case. https://salilab.org/modeller/tutorial/
    
    
4. Are there ions or cofactors present that you don't want to simulate? Probably. Just remove those lines from the pdb file. 
    
    
5. Do you want to keep crystallographic waters? Probably. 
    

6. Visualize your system (PyMol and VMD are my go-tos). See if anything looks wrong. For example, there should only be 1 of your system. In some crystallizations there are repeating units. Unless you intentially want to simulate repeating units (which you probably don't) delete the lines in the PDB file associated with repeats. 
   
    
7. Add 'TER' lines to the PDB. 'TER' separates parts in the PDB file that are not covalently linked. If you have a dimer protein you should have two TER lines after the carboxylic acid terminal oxygens. You don't need to add TER between water molecules, tleap will do that for you.

</p>



### PRO TIP: Always read the paper associated with a crystal structure. The authors will disclose if mutations are present, and provide additional information that may be important to know! 


## 2. tLEAP

This is a program that is used to prepare input for AMBER. Before you do anything with tleap make sure you have adjusted any mutated residues. That way, you won't have to worry about adjusting the hydrogens,just the heavy atoms.


First, protonate your system:

$ > source leaprc.protein.ff14SB #load in a force field$ <br />
$ > source leaprc.water.tip3p #load in a force field$ <br />
$ > x = loadpdb FILENAME.pdb #load in a structure$ <br />
$ > savepdb x FILENAME_PROTONATED.pdb #saves pdb file with everything protonated$ <br />
$ > quit #exit$ <br />

Hydrogens are typically unresolved in crystalization methods. This means the original pdb file will not contain positions of hydrogens. Therefore, you must protonate the system. HOWEVER! I like to protonated my system and then load it into a visualization software to evaluate protonation states, specifically of all HIS residues. Histidine's have two nitrogen atoms that can be protonated, and it will depend on the enrivornment which is more likely the be protonated. It is best practice to manually visualize each HIS residue and declare the protonation state.

<strong> Evaluating HIS protonation states </strong>
1. load protonated pdb into PyMol
2. PyMol> hide nonbonded
3. PyMol> select "his_residues", resn hie #tleap automated chooses the epsilon N to be protonated
4. Show his_residues selection as licorice and color by element differently to see easily 
5. PyMol> show sticks, byres all within 5 of his_residues
6. display sequence
7. open pdb file in text editor
8. start looking at each HIS. If you think the delta N should be protonated, go to that HIS in the pbd and change the HIE to HID for each atom. You also need to delete the HE2 hydrogen atom line. When you load the pdb into tleap again it will detect there needs to be a HD2 atom added, and will do so automatically because you named that residue HID. 


<strong> Back to tleap </strong>

$ > source leaprc.protein.ff14SB #load force field $ <br />
$ > source leaprc.water.TIP3P #load parameters for water$ <br />
$ > x = loadPdb FILE_WITH_ADJUSTED_HIS_STATES.pdb #load protonated pdb file$ <br />
$ > solvatebox x TIP3PBOX 12 #Solvate the complex with a cubic water box $ <br />
$> savepdb x FILENAME_SOLVATED.pdb #save your pdb with waters so you can determine how many waters $ <br />
$ > saveamberparm x temp.prmtop temp.rst7 #doing this right now will tell you if your system has a net charge, and the total number of water molecules$ <br />
$> addions x Na+ Y #Neutralize system, might replace waters $ <br />

#### Pay attention to any errors when loading in your pdb structure. An error likely means tleap isn't sure how to deal with something. Warnings on the other hand are just things tleap wants to bring attention to, but these likely aren't fatal. At this point, tleap will probaly give you a warning statement about an unperturbed charge of the unit. This means your system has a charge, and you probably want a neutral system. This is best done straight in the command line.


### Important: tleap has a tendency to undervalue the water density. It is typically at a density of ~0.85 g/mL when we know water density should be about 0.998 g/mL. This is an annoying feature of tleap. You should keep this in mind when trying to calculate how many ions you need for a particular ionic concentration. When you are running your simulations you will perform some steps in the NPT ensemble which will allow the box volume to equilibrate, resulting in correct density.  

#### If you did the saveamberparm command you will see how many waters your system has. Or you should do the following: In a separate terminal window I use the command "grep -c 'WAT' ~SOMEPATH/FILENAME_SOLVATED.pdb". Divide this number by 3 to get the number of water molecules in your system. 

I have a code that then will determine how many ions you should add: 

$ 1 import numpy as np
  2 import sys
  3
  4
  5 water_residues = sys.argv[1]
  6
  7 desired_ionic_conc = sys.argv[2]
  8
  9
 10 avo = 6.022e23 # molecules/mol
 11 water_mm = 18.01528 #g/mol
 12 water_conversion = 1e-24/water_mm*avo #molecules of water/A^3
 13 water_volume = int(water_residues)/water_conversion #A^3
 14 number_of_ions = float(desired_ionic_conc)/1000/1e24*6.022e23*water_volume #molecules
 15
 16 print('You need to add:',number_of_ions,'ions')$

To run:

python code.py #water_molecules desired_ionic_concentration

Now, back in the tleap window:

$> addions x Na+ XX #However many you need for desired concentration $ <br />
$> addions x Cl- XX #However many you need for desired concentration $ <br />
$> check x #You will probably get Warning:Close contact.. and that's okay.$<br />
$> saveamberparm x FILENAME.prmtop FILENAME.rst7 #Save AMBER topology and coordinate files$ <br />
$> savepdb x FILENAME.pdb #save your pdb with ions and waters $ <br />
$> quit #exit $ <br />

#### tleap will automatically generate a 'leap.log' file. You should keep this! Why not include in your SI so others can see exactly how you prepared your system! It's a pretty large file though... 

### PRO TIP: ALWAYS visualize the system when you think you're all done! 